# Modul 4 - Tugas 2
Pengembangan CRUD fungsional dengan Lambda Expression, Higher-Order Function, Decorator, serta Closure/Currying.

**Fokus**: menjadikan fungsi CRUD sebagai _first class citizens_, memanfaatkan built-in HoF (`map`, `filter`, `reduce`), menambahkan decorator audit, serta menunjukkan closure/currying pada predicate dan updater.

In [ ]:
from pprint import pprint

proyek_awal = {
    "admin": [
        {"id": "SN-001", "tipe": "Sensor Suhu", "nama": "Sensor-Suhu-1", "status": "Aktif"},
        {"id": "AC-001", "tipe": "Aktuator Pendingin", "nama": "Aktuator-AC-1", "status": "Maintenance"},
    ],
    "iot_ops": [
        {"id": "SN-002", "tipe": "Sensor Kelembapan", "nama": "Sensor-Kelembapan-1", "status": "Aktif"},
        {"id": "SN-003", "tipe": "Sensor Cahaya", "nama": "Sensor-Cahaya-1", "status": "Aktif"},
        {"id": "AC-002", "tipe": "Aktuator Ventilasi", "nama": "Aktuator-AC-2", "status": "Aktif"},
    ],
    "guest": [
        {"id": "SN-004", "tipe": "Sensor Tekanan", "nama": "Sensor-Tekan-1", "status": "Nonaktif"}
    ],
}

print("Dataset awal:")
pprint(proyek_awal)

In [ ]:
from functools import reduce
from typing import Callable, Dict, List, Optional

State = Dict[str, List[dict]]
Predicate = Callable[[str, dict], bool]
Updater = Callable[[dict], dict]
Builder = Callable[[], dict]

clone_state = lambda state: {user: [device.copy() for device in devices] for user, devices in state.items()}

def pipe(*ops: Callable[[State], State]) -> Callable[[State], State]:
    return lambda state: reduce(lambda acc, op: op(acc), ops, state)

def audit(action_name: str) -> Callable[[Callable], Callable]:
    def decorator(func: Callable) -> Callable:
        def wrapper(state: State, *args, **kwargs):
            result = func(state, *args, **kwargs)
            print(f"[AUDIT] {action_name} | {func.__name__} | args={args} kwargs={kwargs}")
            return result
        return wrapper
    return decorator

def device_builder(**attrs) -> Builder:
    return lambda: {**attrs}

def curry_set(field: str) -> Callable[[str], Updater]:
    return lambda value: lambda device: {**device, field: value}

def curry_predicate(user: Optional[str] = None, keyword: Optional[str] = None) -> Predicate:
    lowered = keyword.lower() if keyword else None
    return lambda current_user, device: ((current_user == user) if user else True) and (
        (lowered in device["nama"].lower() or lowered in device["tipe"].lower()) if lowered else True
    )

In [ ]:
@audit("CREATE")
def create_device(state: State, user: str, builder: Builder) -> State:
    """Tambahkan device baru tanpa mutasi state awal."""
    new_state = clone_state(state)
    new_state.setdefault(user, []).append(builder())
    return new_state


def read_devices(state: State, predicate: Optional[Predicate] = None) -> List[dict]:
    """Gunakan filter built-in untuk memilih device."""
    predicate = predicate or (lambda *_: True)
    flattened = (
        {**device, "pemilik": user}
        for user, devices in state.items()
        for device in devices
    )
    return list(filter(lambda item: predicate(item["pemilik"], item), flattened))


@audit("UPDATE")
def update_devices(state: State, predicate: Predicate, updater: Updater) -> State:
    """Transformasi device menggunakan map + lambda sesuai predicate."""
    return {
        user: list(
            map(
                lambda device: updater(device) if predicate(user, device) else device,
                devices,
            )
        )
        for user, devices in state.items()
    }


@audit("DELETE")
def delete_devices(state: State, predicate: Predicate) -> State:
    """Hapus device memakai filter built-in."""
    return {
        user: list(filter(lambda device: not predicate(user, device), devices))
        for user, devices in state.items()
    }


def summarize_state(state: State) -> dict:
    """Agregasi menggunakan reduce untuk mendapatkan metrik ringkas."""
    total = reduce(lambda acc, devices: acc + len(devices), state.values(), 0)
    aktif_per_user = {
        user: sum(1 for device in devices if device.get("status", "").lower() == "aktif")
        for user, devices in state.items()
    }
    return {"total_devices": total, "aktif_per_user": aktif_per_user}


def crud_for(user: str):
    """Closure yang menghasilkan CRUD khusus user (Currying implisit)."""
    def create(state: State, builder: Builder) -> State:
        return create_device(state, user, builder)

    def read(state: State, keyword: Optional[str] = None) -> List[dict]:
        return read_devices(state, predicate=curry_predicate(user=user, keyword=keyword))

    def update(state: State, keyword: str, updater: Updater) -> State:
        return update_devices(state, predicate=curry_predicate(user=user, keyword=keyword), updater=updater)

    def delete(state: State, keyword: Optional[str] = None) -> State:
        return delete_devices(state, predicate=curry_predicate(user=user, keyword=keyword))

    return create, read, update, delete

In [ ]:
iot_create, iot_read, iot_update, iot_delete = crud_for("iot_ops")
set_maintenance = curry_set("status")("Maintenance")
sensor_predicate = curry_predicate(user="iot_ops", keyword="sensor")

ops_pipeline = pipe(
    lambda state: iot_create(
        state,
        device_builder(
            id="SN-010", tipe="Sensor Arus", nama="Sensor-Arus-1", status="Aktif"
        ),
    ),
    lambda state: iot_update(state, keyword="sensor", updater=set_maintenance),
    lambda state: iot_delete(state, keyword="Aktuator"),
)

state_setelah_pipeline = ops_pipeline(proyek_awal)
print("State setelah pipeline:")
pprint(state_setelah_pipeline)

In [ ]:
sensor_reader = curry_predicate(keyword="sensor")
aktivasi = curry_set("status")("Aktif")
state_final = update_devices(state_setelah_pipeline, predicate=sensor_reader, updater=aktivasi)

print("Daftar device sensor (read):")
pprint(read_devices(state_final, predicate=sensor_reader))

print("
Ringkasan state:")
pprint(summarize_state(state_final))